# Initailize

In [303]:
from init import *
import json
from pyeoskit import eosapi
# import imp
# imp.reload(evm)

import evm
from evm import get_eth_address_info, w3

eosapi.set_node('http://127.0.0.1:8888')

config.main_token='EOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

priv_keys = [
'5KQwrPbwdL6PhXujxW37FSSQZ1JiwsST4cqQzDeyXtP79zkvFD3',#EOS6MRyAjQq8ud7hVNYcfnVPJqcVpscN5So8BhtHuGYqET5GDW5CV
'5JEcwbckBCdmji5j8ZoMHLEUS8TqQiqBG1DRx1X9DN124GUok9s',#EOS61MgZLN7Frbc2J7giU7JdYjy2TqnfWFjZuLXvpHJoKzWAj7Nst
'5JbDP55GXN7MLcNYKCnJtfKi9aD2HvHAdY7g8m67zFTAFkY1uBB',#EOS5JuNfuZPATy8oPz9KMZV2asKf9m8fb2bSzftvhW55FKQFakzFL
'5K463ynhZoCDDa4RDcr63cUwWLTnKqmdcoTKTHBjqoKfv4u5V7p',#EOS8Znrtgwt8TfpmbVpTKvA2oB8Nqey625CLN8bCN3TEbgx86Dsvr
'5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc',#EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV
'5KT26sGXAywAeUSrQjaRiX9uk9uDGNqC1CSojKByLMp7KRp8Ncw',#EOS8Ep2idd8FkvapNfgUwFCjHBG4EVNAjfUsRRqeghvq9E91tkDaj
]
for priv_key in priv_keys:
    wallet.import_key('test', priv_key)

db.reset()

evm.set_chain_id(888)
print('done!')

done!


# Add eosio.code to active permission

In [304]:
account_name = 'helloworld11'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":account_name,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')

done!


# Deploy evm smart contract to network

In [312]:
try:
    eosapi.clear_abi_cache('helloworld11');
except Exception as e:
    print(e)

source_path = 'ethereum_vm'
#source_path = '/Users/newworld/dev/eos-with-evm/contracts/ethereum_vm'

evm_code_file = os.path.join(source_path, 'ethereum_vm.cpp')
deploy_cpp_contract_from_file('helloworld11', evm_code_file, includes = [source_path])
print('Done!')

update contract
Done!


In [313]:
main_account = 'helloworld11'
test_account = 'helloworld12'

args = {'chainid': 888}
try:
    r = eosapi.push_action(main_account, 'setchainid', args, {main_account:'active'})
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

1795


# Create an ETH account and bind it to an EOS account

In [314]:

eth_address = None
name = 'helloworld11'
args = {'account':'helloworld12', 'text': 'hello,world'}
try:
    r = eosapi.push_action(name, 'create', args, {'helloworld12':'active'})
    eth_address = r['processed']['action_traces'][0]['receipt']['return_value']
    print('+++', eth_address)
#    eth_address = w3.toChecksumAddress(eth_address)
    print('eth address:', eth_address)
    print('++++console:', r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    if hasattr(e, 'response'):
        parsed = json.loads(e.response)
        print('+++error:\n', json.dumps(parsed, indent=4))
    else:
        print(e)

+++ f375c0d7e20a15b3d35af771763ca166d54cce97
eth address: 0xf375C0D7E20a15b3D35AF771763CA166D54cce97
++++console: f375c0d7e20a15b3d35af771763ca166d54cce97
24073


### Verify eth address method 1

In [321]:
import rlp
import hashlib
from eth_utils import keccak
e = rlp.encode(['helloworld12', 'hello,world'])
h = keccak(e)
if eth_address:
    print(h[12:].hex())
    assert eth_address == h[12:].hex()
else:
    eth_address = h[12:].hex()
    print(eth_address)

f375c0d7e20a15b3d35af771763ca166d54cce97


### Verify result method 2

In [30]:
#install pysah3
%system python3.7 -m pip install pysha3

['Requirement already satisfied: pysha3 in /Users/newworld/opt/anaconda3/lib/python3.7/site-packages (1.0.2)']

In [322]:
import rlp
import sha3
e = rlp.encode(['helloworld12', 'hello,world'])
h = sha3.keccak_256()
h.update(e)
h = h.digest()
if eth_address:
    assert eth_address == h[12:].hex()
else:
    eth_address = h[12:].hex()
    print(eth_address)

In [323]:
eth_address = w3.toChecksumAddress(eth_address)
print(eth_address)

0xf375C0D7E20a15b3D35AF771763CA166D54cce97


# Deposit Test

In [324]:
r = eosapi.transfer('helloworld12', 'helloworld11', 10.0, 'deposit')

In [325]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 1, 'balance': '10.0000 SYS'}

In [326]:
r = eosapi.transfer('helloworld12', 'helloworld11', 1.0, 'deposit')

In [327]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 1, 'balance': '11.0000 SYS'}

# Withdraw Test

In [328]:
eosapi.get_balance('helloworld12')

9999989.0

### Check balance in ETH account

In [329]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 1, 'balance': '11.0000 SYS'}

### Withdraw 1.0000

In [330]:
name = 'helloworld12'
args = {'account': name, 'amount': '1.0000 SYS'}
try:
    r = eosapi.push_action('helloworld11', 'withdraw', args, {name:'active'})
    print('++++console:', r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

++++console: 4,SYS4,SYS
930


### Check balance

In [331]:
eosapi.get_balance('helloworld12')

9999990.0

### Check balance in binded ETH account

In [332]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 1, 'balance': '10.0000 SYS'}

bind an eth address through directly
```python
name = 'helloworld11'
args = bytes.fromhex('b654A7A81E0aeb7721A22F27A04ecf5aF0E8a9A3')
try:
    r = eosapi.push_action(name, 'activate', args, {name:'active'})
    print('++++console:', r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)
```

# Deploy an ethereum smart contract

In [333]:
from solcx import compile_source, compile_files

greeter = '''pragma solidity ^0.6.0;
contract Greeter {
    uint value;
    event onSetValue(uint value);
    event onGetValue(uint value);
    constructor() public {
        value = 1;
    }

    function getValue2() payable public returns (uint){
//        msg.sender.transfer(1000);
        emit onGetValue(value);
        return value;
    }

    function getValue() payable public returns (string memory){
        return "hello,world";
    }

    function setValue(uint v) payable public {
        value = v;
        emit onSetValue(v);
    }
}
'''

def compile(contract_source_code, main_class):
    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol[main_class]

    return contract_interface


main_class = '<stdin>:Greeter'
contract_source_code = greeter
contract_interface = compile(contract_source_code, main_class)
bytecode = contract_interface['bin']
abi = contract_interface['abi']
print('Done!')

Done!


In [334]:
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)
ret = Greeter.constructor().transact({'from':'0xf375C0D7E20a15b3D35AF771763CA166D54cce97'})
logs = ret['processed']['action_traces'][0]['console']
print('logs:', logs)
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
#print('+++++output:', output)
output = json.loads(output)
print(output)
contract_address = w3.toChecksumAddress(output['new_address'])
print(contract_address)

logs: []
{'new_address': '0633a42c777f64f895de1b0097de00c8d181a5e9', 'output': '6080604052600436106100345760003560e01c8063209652551461003957806355241077146100bc578063c515205d146100ea575b600080fd5b610041610108565b6040518080602001828103825283818151815260200191508051906020019080838360005b83811015610081578082015181840152602081019050610066565b50505050905090810190601f1680156100ae5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6100e8600480360360208110156100d257600080fd5b8101908080359060200190929190505050610145565b005b6100f2610186565b6040518082815260200191505060405180910390f35b60606040518060400160405280600b81526020017f68656c6c6f2c776f726c64000000000000000000000000000000000000000000815250905090565b806000819055507ff48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773816040518082815260200191505060405180910390a150565b60007f29325e60ff39fad605f9bb4f2fd7cc0ee8e778b3f52cb561c2d8eebfcec4ca4b6000546040518082815260200191505060405180910390a160005490509056fea264

#### Call setValue in ethereum smart contract

In [335]:
contract_address

'0x0633A42c777f64f895dE1B0097de00C8D181A5e9'

In [336]:
args = {'from': eth_address,'to':contract_address}
ret = Greeter.functions.setValue(0xffaabbcc).transact(args)
logs = ret['processed']['action_traces'][0]['console']
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
output = json.loads(output)
print('+++logs:', logs)
print('+++output:', output)

+++logs: [{"address":"0633a42c777f64f895de1b0097de00c8d181a5e9","data" :"5524107700000000000000000000000000000000000000000000000000000000ffaabbcc","topics" :["f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773"]}]
+++output: {'new_address': '0000000000000000000000000000000000000000', 'output': ''}


# transfer test

In [338]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 3, 'balance': '10.0000 SYS'}

In [339]:
get_eth_address_info('helloworld11', contract_address)

{'nonce': 2, 'balance': '0.0000 SYS'}

### setValue and transfer 1.0 to contract address

In [340]:
print('transfer from ', eth_address, 'to', contract_address)
args = {'from': eth_address,'to':contract_address, 'value':10000}
ret = Greeter.functions.setValue(0xffaabbcc).transact(args)
logs = ret['processed']['action_traces'][0]['console']
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
output = json.loads(output)
print(logs)
print(output)

transfer from  0xf375C0D7E20a15b3D35AF771763CA166D54cce97 to 0x0633A42c777f64f895dE1B0097de00C8D181A5e9
[{"address":"0633a42c777f64f895de1b0097de00c8d181a5e9","data" :"5524107700000000000000000000000000000000000000000000000000000000ffaabbcc","topics" :["f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773"]}]
{'new_address': '0000000000000000000000000000000000000000', 'output': ''}


### get address info

In [341]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 4, 'balance': '9.0000 SYS'}

In [342]:
get_eth_address_info('helloworld11', contract_address)

{'nonce': 2, 'balance': '1.0000 SYS'}

In [343]:
from solcx import compile_source, compile_files

greeter = '''pragma solidity ^0.6.0;
contract Greeter {
    uint value;
    event onSetValue(uint value);
    event onGetValue(uint value);
    event onTransferBack(uint remainBalance);
    
    constructor() public {
        value = 1;
    }

    function getValue2() payable public returns (uint){
//        msg.sender.transfer(1000);
        emit onGetValue(value);
        return value;
    }

    function getValue() payable public returns (string memory){
        return "hello,world";
    }

    function setValue(uint v) payable public {
        require(block.gaslimit == 0x7fffffffffffffff);
        require(block.coinbase == 0x0633A42c777f64f895dE1B0097de00C8D181A5e9);
        require(block.difficulty == 0x020000);
        require(blockhash(0) == 0);
        require(ecrecover(0, 0, 0, 0) == address(0));
        emit onSetValue(gasleft());
    }

    function transferBack() payable public {
        uint256 oldBalance = msg.sender.balance;
        msg.sender.transfer(10000);
        uint256 newBalance = msg.sender.balance;
        require(newBalance - oldBalance == 10000);
    }
}
'''

def compile(contract_source_code, main_class):
    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol[main_class]

    return contract_interface


main_class = '<stdin>:Greeter'
contract_source_code = greeter
contract_interface = compile(contract_source_code, main_class)
bytecode = contract_interface['bin']
abi = contract_interface['abi']
print('Done!')


Done!


In [344]:
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)
ret = Greeter.constructor().transact({'from':'0xf375C0D7E20a15b3D35AF771763CA166D54cce97'})
logs = ret['processed']['action_traces'][0]['console']
print('logs:', logs)
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
#print('+++++output:', output)
output = json.loads(output)
#print(output)
contract_address = w3.toChecksumAddress(output['new_address'])
print('+++contract_address:', contract_address)

logs: []
+++contract_address: 0x57a25B5e534745319C40969A7448c01d81350E7d


In [345]:
args = {'from': eth_address,'to':contract_address}
ret = Greeter.functions.setValue(0xffaabbcc).transact(args)
logs = ret['processed']['action_traces'][0]['console']
print('+++logs:', logs)

+++logs: [{"address":"57a25b5e534745319c40969a7448c01d81350e7d","data" :"5524107700000000000000000000000000000000000000000000000000000000ffaabbcc","topics" :["f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773"]}]


In [346]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 6, 'balance': '9.0000 SYS'}

In [347]:
get_eth_address_info('helloworld11', contract_address)

{'nonce': 2, 'balance': '0.0000 SYS'}

In [348]:
args = {'from': eth_address,'to':contract_address, 'value': 10000} # 1.0 SYS
ret = Greeter.functions.setValue(0xffaabbcc).transact(args)
logs = ret['processed']['action_traces'][0]['console']
print(ret['processed']['elapsed'])
print('+++logs:', logs)

843
+++logs: [{"address":"57a25b5e534745319c40969a7448c01d81350e7d","data" :"5524107700000000000000000000000000000000000000000000000000000000ffaabbcc","topics" :["f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773"]}]


In [349]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 7, 'balance': '8.0000 SYS'}

In [350]:
get_eth_address_info('helloworld11', contract_address)

{'nonce': 2, 'balance': '1.0000 SYS'}

# test transfer from contract address back

In [351]:
args = {'from': eth_address,'to':contract_address} # 1.0 SYS
ret = Greeter.functions.transferBack().transact(args)
logs = ret['processed']['action_traces'][0]['console']
print(ret['processed']['elapsed'])

print('+++logs:', logs)

574
+++logs: []


In [352]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 8, 'balance': '9.0000 SYS'}

In [353]:
get_eth_address_info('helloworld11', contract_address)

{'nonce': 2, 'balance': '0.0000 SYS'}

# 1. The Application MUST persist an “Account Table” consisting of:
> * A unique 160bit account ID
> * A nonce (sequence number)
> * An EOSIO token balance (aka SYS)
> * [optionally,] a unique associated EOSIO account


# 2. The Application MUST persist an “Account State Table” per account, if it would not be empty, consisting of:
> * A unique 256bit key
> * A 256bit value


# 3.The Application MUST persist an “Account Code Table” per account, if it would not be empty, consisting of:
EVM bytecode associated with account

### 4. The Application MUST execute EVM transactions as faithfully to the Ethereum Yellow Paper as Possible, with the following specific requirements:

1. There will be no effective BLOCK gas limit. Instructions that return a BLOCK limit should return a  sufficiently large supply
check code at:
[evm4eos/main.cpp#L108](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L108)

2. The TRANSACTION gas limit will be enforced
check code at [evm4eos/main.cpp#L83](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L83)
and [evm4eos/main.cpp#L169](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L169)

3. The sender WILL NOT be billed for the gas, the gas price MAY therefore be locked at some suitable value.

check code at [evm4eos/main.cpp#L83](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L83)
and [evm4eos/main.cpp#L169](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L169)

4. All other gas mechanics/instructions should be maintained

5. Block number and timestamp should represent the native EOSIO block number and time

check code at [main.cpp#L150](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L150)

6. Block hash, coinbase, and difficulty should return static values


check code at [evm4eos/EosExtVM.cpp#L188](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/EosExtVM.cpp#L188)
and
[libevm2/LegacyVM.cpp#L1396](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/libevm2/LegacyVM.cpp#L1396)
and
[main.cpp#L111](https://github.com/learnforpractice/aleth/blob/5620dbe19ab5d8b4d7b0e9be77bf45eb461d49d2/evm4eos/main.cpp#L111)


# 5. The Application MUST implement an action named “raw”:

Whose inputs are:
> * a binary Ethereum transaction encoded as it appears in a serialized Ethereum block
[optionally,]
> * a 160bit account identifier “Sender”

Which results in the:
> * Appropriate Updates to Account, Account State, and Account Code Tables reflecting the application of the transaction
> * Log output (via EOSIO print intrinsics)

If the “R” and “S” values of the transaction are NOT 0:
> * A transaction containing this action must fail if the signature (V, R, S) within the input does not recover to a valid and known 160bit account identifier in the Accounts Table

If the “R” and “S” values of the transaction are 0:
> * A transaction containing this action must fail if “Sender” input parameter is not present or does not refer to a valid and known 160bit account identifier in the Accounts Table

> * If the associated entry in the Accounts Table has no Associated EOSIO Account OR if the transaction has not been authorized by the Associated EOSIO Account

# 6. The Application MUST implement an action named “create”:

Whose inputs are:
> * An EOSIO account
> * An arbitrary length string

Which results in new Account Table entry with:
> * Balance = 0
> * Nonce = 1
> * Account identifier = the rightmost 160 bits of the Keccak hash of the RLP encoding of the structure containing only the EOSIO account name and the arbitrary input string.

A transaction containing this action must fail if it is not authorized by the EOSIO account listed in the inputs

A transaction containing this action must fail if an Account Table Entry exists with this EOSIO account associated.


# 7. The Application MUST respond to EOSIO token transfers:
7.1 Provided that the EOSIO account in the “from” field of the transfer maps to a known and valid Account Table entry through the entry’s unique Associated EOSIO account

7.2 Transferred tokens should be added to the Account Table entry’s balance

# 8. The Application MUST implement an action named “withdraw”:
Whose inputs are:
> * An EOSIO account
> * A token amount

Which results in:
> * Deducting the amount from the associated Account Table Entry’s balance
> * Sending an inline EOSIO token transfer for the amount to the EOSIO account

A transaction containing this action must fail if it is not authorized by the EOSIO account listed in the inputs OR if such a withdrawal would leave the Account Table Entry’s balance negative.






# 9. The Application MAY implement additional actions for maintenance or transaction processing so long as they do not violate the key principles of the execution model above.

# 10. The Application MUST implement some method of specifying the “CHAIN_ID” for EIP-155 compatibility:
This MAY be done at compile time
This MAY be done with an additional initialization action